In [1]:
import pandas as pd
import numpy as np
import optuna 
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner
from sklearn.model_selection import KFold
from catboost import Pool
import catboost as cb
from sklearn.metrics import mean_squared_log_error

In [2]:
train = pd.read_csv('/kaggle/input/playground-series-s5e5/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s5e5/test.csv')

In [3]:
sex_mapping = {
    'male':0,
    'female':1
}
train['Sex'] = train['Sex'].map(sex_mapping)
test['Sex'] = test['Sex'].map(sex_mapping)
X = train.drop(columns=['Calories', 'id'])
y = train['Calories']

In [4]:
seed = 1

# catboost

In [5]:
# def cat_objective(trial):
#     # Parameter definitions
#     params = {
#         'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.2, log=True),
#         'depth': trial.suggest_int('depth', 4, 13),  # Shallower trees for GPU
#         'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-4, 10.0, log=True),
#         'random_strength': trial.suggest_float('random_strength', 1e-4, 3.0),
#         'grow_policy': trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise']),
#         'bootstrap_type': trial.suggest_categorical('bootstrap_type', ['Bayesian', 'Bernoulli']),
#         'iterations': trial.suggest_int('iterations', 1000, 3000),
#         'task_type': 'GPU',
#         'devices': [0],
#         'verbose': False,
#         'random_seed': 42,
#         'early_stopping_rounds': 50,
#     }
    
#     # GPU-specific adjustments
#     params.update({
#         'border_count': trial.suggest_int('border_count', 64, 128),
#         'has_time': trial.suggest_categorical('has_time', [True, False])
#     })
    
#     # Handle bootstrap types
#     if params['bootstrap_type'] == 'Bernoulli':
#         params['subsample'] = trial.suggest_float('subsample', 0.8, 1.0)
    
#     # Grow policy constraints
#     params['min_data_in_leaf'] = trial.suggest_int('min_data_in_leaf', 20, 100)
    
#     kf = KFold(n_splits=5, shuffle=True, random_state=42)  # Fixed seed reference
#     scores = []
    
#     for fold, (train_idx, valid_idx) in enumerate(kf.split(X, y)):
#         X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
#         y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
        
#         y_train_log = np.log1p(y_train)
        
#         train_pool = Pool(X_train, y_train_log)
#         valid_pool = Pool(X_valid)
        
#         model = cb.CatBoostRegressor(**params)
#         model.fit(train_pool,
#                  early_stopping_rounds=params['early_stopping_rounds'])
        
#         preds_log = model.predict(X_valid)
#         preds = np.expm1(preds_log)
#         preds = np.clip(preds, 1e-6, None)
        
#         rmsle = np.sqrt(mean_squared_log_error(y_valid, preds))
#         scores.append(rmsle)
        
#         # Report intermediate score
#         trial.report(rmsle, step=fold)
        
#         # DELAYED PRUNING: Only check for pruning after 3rd fold (index 2)
#         if fold >= 2 and trial.should_prune():
#             print(f"Trial {trial.number} pruned after fold {fold + 1}")
#             raise optuna.TrialPruned()
    
#     final_score = np.mean(scores)
#     print(f"Trial {trial.number} completed with score: {final_score:.6f}")
#     return final_score

# # Create study with custom pruner settings
# study = optuna.create_study(
#     direction='minimize',
#     sampler=TPESampler(seed=42, multivariate=True),
#     # Use MedianPruner with delayed start
#     pruner=MedianPruner(
#         n_startup_trials=10,     # Don't prune first 10 trials
#         n_warmup_steps=2,        # Wait for 3 folds (0,1,2) before pruning
#         interval_steps=1         # Check pruning after each fold
#     )
# )

# # Alternative: More conservative pruning
# # study = optuna.create_study(
# #     direction='minimize',
# #     sampler=TPESampler(seed=42, multivariate=True),
# #     pruner=MedianPruner(
# #         n_startup_trials=15,     # Don't prune first 15 trials  
# #         n_warmup_steps=3,        # Wait for 4 folds before pruning
# #         interval_steps=1
# #     )
# # )

# # Even more conservative: Custom percentile pruner
# # from optuna.pruners import PercentilePruner
# # study = optuna.create_study(
# #     direction='minimize',
# #     sampler=TPESampler(seed=42, multivariate=True),
# #     pruner=PercentilePruner(
# #         percentile=25.0,         # Only prune bottom 25% of trials
# #         n_startup_trials=20,     # More startup trials
# #         n_warmup_steps=3         # Wait for 4 folds
# #     )
# # )

# print("Starting optimization with delayed pruning...")
# print("Pruning will only occur after fold 3 (index 2)")

# # Run optimization
# study.optimize(cat_objective, n_trials=150)

# # Print results
# print(f"\nBest trial: {study.best_trial.number}")
# print(f"Best score: {study.best_value:.6f}")
# print("Best parameters:")
# for key, value in study.best_params.items():
#     print(f"  {key}: {value}")

# # Analyze pruning statistics
# pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
# completed_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]

# print(f"\nPruning Statistics:")
# print(f"Total trials: {len(study.trials)}")
# print(f"Completed trials: {len(completed_trials)}")
# print(f"Pruned trials: {len(pruned_trials)}")
# print(f"Pruning rate: {len(pruned_trials)/len(study.trials)*100:.1f}%")

# # Show when trials were pruned
# if pruned_trials:
#     pruning_folds = [len(t.intermediate_values) for t in pruned_trials]
#     print(f"Pruned after folds: {pruning_folds}")
#     print(f"Average fold when pruned: {np.mean(pruning_folds):.1f}")

# # # Alternative implementation with manual pruning control
# # def cat_objective_manual_pruning(trial):
# #     """Version with complete manual control over pruning"""
# #     # ... [same parameter setup as above] ...
    
# #     # Your parameter setup code here (same as above)
# #     params = {
# #         'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.2, log=True),
# #         'depth': trial.suggest_int('depth', 4, 13),
# #         'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-4, 10.0, log=True),
# #         'random_strength': trial.suggest_float('random_strength', 1e-4, 3.0),
# #         'grow_policy': trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise']),
# #         'bootstrap_type': trial.suggest_categorical('bootstrap_type', ['Bayesian', 'Bernoulli']),
# #         'iterations': trial.suggest_int('iterations', 1000, 3000),
# #         'task_type': 'GPU',
# #         'devices': [0],
# #         'verbose': False,
# #         'random_seed': 42,
# #         'early_stopping_rounds': 50,
# #     }
    
# #     if params['bootstrap_type'] == 'Bernoulli':
# #         params['subsample'] = trial.suggest_float('subsample', 0.8, 1.0)
    
# #     params.update({
# #         'border_count': trial.suggest_int('border_count', 64, 128),
# #         'has_time': trial.suggest_categorical('has_time', [True, False]),
# #         'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 20, 100)
# #     })
    
# #     kf = KFold(n_splits=5, shuffle=True, random_state=42)
# #     scores = []
    
# #     # Define your pruning policy
# #     MIN_FOLDS_BEFORE_PRUNING = 3  # Change this to 4 if you want to wait longer
    
# #     for fold, (train_idx, valid_idx) in enumerate(kf.split(X, y)):
# #         # ... [same training code] ...
# #         X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
# #         y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
        
# #         y_train_log = np.log1p(y_train)
# #         train_pool = Pool(X_train, y_train_log)
        
# #         model = cb.CatBoostRegressor(**params)
# #         model.fit(train_pool, early_stopping_rounds=params['early_stopping_rounds'])
        
# #         preds_log = model.predict(X_valid)
# #         preds = np.expm1(preds_log)
# #         preds = np.clip(preds, 1e-6, None)
        
# #         rmsle = np.sqrt(mean_squared_log_error(y_valid, preds))
# #         scores.append(rmsle)
# #         trial.report(rmsle, step=fold)
        
# #         # Custom pruning logic: only prune after minimum folds AND if performance is bad
# #         if fold >= MIN_FOLDS_BEFORE_PRUNING:
# #             current_avg = np.mean(scores)
# #             # Additional condition: only prune if significantly worse than median
# #             if trial.should_prune():
# #                 print(f"Trial {trial.number} pruned after fold {fold + 1} with avg score {current_avg:.6f}")
# #                 raise optuna.TrialPruned()
    
# #     return np.mean(scores)

# # Uncomment to use the manual version:
# # study_manual = optuna.create_study(
# #     direction='minimize',
# #     sampler=TPESampler(seed=42, multivariate=True),
# #     pruner=MedianPruner(n_startup_trials=15, n_warmup_steps=3)
# # )
# # study_manual.optimize(cat_objective_manual_pruning, n_trials=150)

In [6]:
# print("Best trial:")
# trial = study.best_trial
# print(f"  RMSLE: {trial.value:.5f}")
# print("  Params: ")
# for key, value in trial.params.items():
#     print(f"    {key}: {value}")

# # Train final model
# best_params = study.best_params.copy()
# best_params.update({'early_stopping_rounds': None}) 

# Lightgbm

In [7]:
# import lightgbm as lgb
# import warnings

# warnings.filterwarnings('ignore')
# def lgbm_objective(trial):
#     # Parameter definitions with very diverse ranges
#     params = {
#         'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.3, log=True),
#         'num_leaves': trial.suggest_int('num_leaves', 10, 300),
#         'max_depth': trial.suggest_int('max_depth', 3, 15),
#         'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 5, 200),
#         'min_sum_hessian_in_leaf': trial.suggest_float('min_sum_hessian_in_leaf', 1e-4, 10.0, log=True),
#         'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
#         'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
#         'bagging_freq': trial.suggest_int('bagging_freq', 0, 7),
#         'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
#         'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
#         'min_gain_to_split': trial.suggest_float('min_gain_to_split', 0.0, 0.5),
#         'max_bin': trial.suggest_int('max_bin', 63, 255),
#         'num_iterations': trial.suggest_int('num_iterations', 500, 3000),
#         'boosting_type': trial.suggest_categorical('boosting_type', ['gbdt']),
#         'objective': 'regression',
#         'metric': 'rmse',
#         'verbose': -1,
#         'random_seed': 42,
#         'early_stopping_rounds': 50,
#         'device_type': 'gpu',
#         # 'tree_method': 'gpu_hist',
#         # 'device': 'cuda',
#         # 'gpu_platform_id': 0,
#         # 'gpu_device_id': 0,
#     }
    
    
#     # Additional diverse parameters
#     params['path_smooth'] = trial.suggest_float('path_smooth', 0.0, 100.0)
#     params['extra_trees'] = trial.suggest_categorical('extra_trees', [True, False])
#     if params['extra_trees']:
#         params['extra_seed'] = trial.suggest_int('extra_seed', 1, 1000)
    
#     kf = KFold(n_splits=5, shuffle=True, random_state=42)
#     scores = []
    
#     for fold, (train_idx, valid_idx) in enumerate(kf.split(X, y)):
#         X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
#         y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
        
#         y_train_log = np.log1p(y_train)
        
#         train_data = lgb.Dataset(X_train, label=y_train_log)
#         valid_data = lgb.Dataset(X_valid, label=np.log1p(y_valid), reference=train_data)
        
#         model = lgb.train(
#             params,
#             train_data,
#             valid_sets=[valid_data],
#             callbacks=[lgb.early_stopping(params['early_stopping_rounds']), lgb.log_evaluation(0)]
#         )
        
#         preds_log = model.predict(X_valid, num_iteration=model.best_iteration)
#         preds = np.expm1(preds_log)
#         preds = np.clip(preds, 1e-6, None)
        
#         rmsle = np.sqrt(mean_squared_log_error(y_valid, preds))
#         scores.append(rmsle)
        
#         # Report intermediate score
#         trial.report(rmsle, step=fold)
        
#         # DELAYED PRUNING: Only check for pruning after 3rd fold (index 2)
#         if fold >= 2 and trial.should_prune():
#             print(f"Trial {trial.number} pruned after fold {fold + 1}")
#             raise optuna.TrialPruned()
    
#     final_score = np.mean(scores)
#     print(f"Trial {trial.number} completed with score: {final_score:.6f}")
#     return final_score

# # Create study for LightGBM
# study_lgbm = optuna.create_study(
#     direction='minimize',
#     sampler=TPESampler(seed=42, multivariate=True),
#     pruner=MedianPruner(
#         n_startup_trials=10,
#         n_warmup_steps=2,
#         interval_steps=1
#     )
# )

# print("Starting LightGBM optimization with delayed pruning...")
# print("Pruning will only occur after fold 3 (index 2)")
# study_lgbm.optimize(lgbm_objective, n_trials=150)

In [8]:
# print("Best trial:")
# trial = study_lgbm.best_trial
# print(f"  RMSLE: {trial.value:.5f}")
# print("  Params: ")
# for key, value in trial.params.items():
#     print(f"    {key}: {value}")

# # Train final model
# best_params = study_lgbm.best_params.copy()
# best_params.update({'early_stopping_rounds': None}) 

# xgboost

In [9]:
import xgboost as xgb

def xgb_objective(trial):
    # Parameter definitions with very diverse ranges
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_child_weight': trial.suggest_float('min_child_weight', 0.1, 20.0, log=True),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.4, 1.0),
        'colsample_bynode': trial.suggest_float('colsample_bynode', 0.4, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'max_delta_step': trial.suggest_float('max_delta_step', 0.0, 10.0),
        'n_estimators': trial.suggest_int('n_estimators', 500, 5000),
        'booster': trial.suggest_categorical('booster', ['gbtree']),
        'tree_method': 'gpu_hist',
        'gpu_id': 0,
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'verbosity': 0,
        'random_state': 42,
        'early_stopping_rounds': 50,
    }
    
    # Booster specific parameters
    if params['booster'] == 'dart':
        params['sample_type'] = trial.suggest_categorical('sample_type', ['uniform', 'weighted'])
        params['normalize_type'] = trial.suggest_categorical('normalize_type', ['tree', 'forest'])
        params['rate_drop'] = trial.suggest_float('rate_drop', 0.01, 0.5)
        params['one_drop'] = trial.suggest_categorical('one_drop', [0, 1])
        params['skip_drop'] = trial.suggest_float('skip_drop', 0.0, 0.8)
    elif params['booster'] == 'gblinear':
        params['feature_selector'] = trial.suggest_categorical('feature_selector', ['cyclic', 'shuffle', 'random'])
    else:  # gbtree
        params['grow_policy'] = trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide'])
        if params['grow_policy'] == 'lossguide':
            params['max_leaves'] = trial.suggest_int('max_leaves', 0, 256)
        params['max_bin'] = trial.suggest_int('max_bin', 64, 512)
    
    # Additional diverse parameters
    params['scale_pos_weight'] = trial.suggest_float('scale_pos_weight', 0.1, 10.0)
    params['max_cat_to_onehot'] = trial.suggest_int('max_cat_to_onehot', 1, 10)
    
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = []
    
    for fold, (train_idx, valid_idx) in enumerate(kf.split(X, y)):
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
        
        y_train_log = np.log1p(y_train)
        y_valid_log = np.log1p(y_valid)
        
        model = xgb.XGBRegressor(**params)
        model.fit(
            X_train, y_train_log,
            eval_set=[(X_valid, y_valid_log)],
            verbose=False
        )
        
        preds_log = model.predict(X_valid)
        preds = np.expm1(preds_log)
        preds = np.clip(preds, 1e-6, None)
        
        rmsle = np.sqrt(mean_squared_log_error(y_valid, preds))
        scores.append(rmsle)
        
        # Report intermediate score
        trial.report(rmsle, step=fold)
        
        # DELAYED PRUNING: Only check for pruning after 3rd fold (index 2)
        if fold >= 1 and trial.should_prune():
            print(f"Trial {trial.number} pruned after fold {fold + 1}")
            raise optuna.TrialPruned()
    
    final_score = np.mean(scores)
    print(f"Trial {trial.number} completed with score: {final_score:.6f}")
    return final_score

# Create study for XGBoost
study_xgb = optuna.create_study(
    direction='minimize',
    sampler=TPESampler(seed=42, multivariate=True),
    pruner=MedianPruner(
        n_startup_trials=10,
        n_warmup_steps=2,
        interval_steps=1
    )
)

print("Starting XGBoost optimization with delayed pruning...")
print("Pruning will only occur after fold 3 (index 2)")
study_xgb.optimize(xgb_objective, n_trials=150)

/usr/local/lib/python3.11/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2025-05-28 08:29:32,723] A new study created in memory with name: no-name-3011e00d-6175-43f3-ab62-36ff7fe4c698


Starting XGBoost optimization with delayed pruning...
Pruning will only occur after fold 3 (index 2)


[I 2025-05-28 08:33:13,214] Trial 0 finished with value: 0.060847950453070246 and parameters: {'learning_rate': 0.023171758042400847, 'max_depth': 15, 'min_child_weight': 4.83437145318464, 'subsample': 0.759195090518222, 'colsample_bytree': 0.4936111842654619, 'colsample_bylevel': 0.49359671220172163, 'colsample_bynode': 0.4348501673009197, 'reg_alpha': 0.6245760287469893, 'reg_lambda': 0.002570603566117598, 'gamma': 0.004619347374377372, 'max_delta_step': 0.20584494295802447, 'n_estimators': 4865, 'booster': 'gbtree', 'grow_policy': 'depthwise', 'max_bin': 145, 'scale_pos_weight': 1.915704647548995, 'max_cat_to_onehot': 4}. Best is trial 0 with value: 0.060847950453070246.


Trial 0 completed with score: 0.060848


[I 2025-05-28 08:35:03,185] Trial 1 finished with value: 0.0605602881329176 and parameters: {'learning_rate': 0.042861395747190736, 'max_depth': 8, 'min_child_weight': 0.4678719265016201, 'subsample': 0.7671117368334277, 'colsample_bytree': 0.4836963163912251, 'colsample_bylevel': 0.5752867891211308, 'colsample_bynode': 0.619817105976215, 'reg_alpha': 0.00012724181576752517, 'reg_lambda': 0.1165691561324743, 'gamma': 3.9572205641009174e-07, 'max_delta_step': 5.142344384136116, 'n_estimators': 3166, 'booster': 'gbtree', 'grow_policy': 'lossguide', 'max_leaves': 43, 'max_bin': 93, 'scale_pos_weight': 9.493966818808, 'max_cat_to_onehot': 10}. Best is trial 1 with value: 0.0605602881329176.


...

Trial 147 pruned after fold 5


[I 2025-05-28 13:26:53,807] Trial 148 pruned. 


Trial 148 pruned after fold 3


[I 2025-05-28 13:27:32,683] Trial 149 pruned. 


Trial 149 pruned after fold 3


In [10]:
print("Best trial:")
trial = study_xgb.best_trial
print(f"  RMSLE: {trial.value:.5f}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

# Train final model
best_params = study_xgb.best_params.copy()
best_params.update({'early_stopping_rounds': None}) 

Best trial:
  RMSLE: 0.05942
  Params: 
    learning_rate: 0.005259384953220211
    max_depth: 10
    min_child_weight: 0.10230593448413597
    subsample: 0.49101690286763244
    colsample_bytree: 0.9934807429906339
    colsample_bylevel: 0.7565299575549633
    colsample_bynode: 0.9632535324582226
    reg_alpha: 0.06388723332518843
    reg_lambda: 1.086396993136656e-07
    gamma: 1.4437051685678297e-07
    max_delta_step: 7.229720505305216
    n_estimators: 3234
    booster: gbtree
    grow_policy: depthwise
    max_bin: 130
    scale_pos_weight: 3.0014652523374097
    max_cat_to_onehot: 5
